In [1]:
import pandas as pd
import json

In [2]:
raw = pd.read_csv('../../data/individuals.csv')
raw.head()

,新No.,旧No.,確定年,確定月,確定日,年代,性別,居住地1,居住地2
0,1,1,2020,1,15,30代,男,神奈川県,NaN
1,2,2,2020,1,24,40代,男,中国（武漢市）,NaN
2,3,3,2020,1,25,30代,女,中国（武漢市）,NaN
3,4,4,2020,1,26,40代,男,中国（武漢市）,NaN
4,5,5,2020,1,28,40代,男,中国（武漢市）,NaN


In [3]:
region_to_prefecture = {
    '北海道': ['北海道'],
    '東北': ['青森県', '岩手県', '秋田県', '宮城県', '山形県', '福島県'],
    '関東': ['茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県'],
    '中部': ['新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県', '静岡県', '愛知県'],
    '近畿': ['三重県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県'],
    '中国': ['鳥取県', '島根県', '岡山県', '広島県', '山口県'],
    '四国': ['徳島県', '香川県', '愛媛県', '高知県'],
    '九州沖縄': ['福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'],
}
prefectures = []
for k,v in region_to_prefecture.items():
    prefectures += v

In [48]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

In [4]:
raw['date'] = pd.to_datetime(raw.apply(lambda r: f"{r['確定年']}-{r['確定月']:02d}-{r['確定日']:02d}", axis=1))

In [5]:
count_by_location = raw[['date', '居住地1','新No.']].pivot_table(index='date',columns='居住地1',values='新No.',aggfunc='count')
count_by_location.loc['2020-03-01':,'北海道']

date
2020-03-01     2.0
2020-03-02     5.0
2020-03-03     2.0
2020-03-04     4.0
2020-03-05     NaN
2020-03-06     7.0
2020-03-07     8.0
2020-03-08     3.0
2020-03-09     7.0
2020-03-10     3.0
2020-03-11     7.0
2020-03-12    10.0
2020-03-13     8.0
2020-03-14     6.0
2020-03-15     6.0
2020-03-16     4.0
2020-03-17     NaN
2020-03-18     2.0
2020-03-19     3.0
Name: 北海道, dtype: float64

In [6]:
s = count_by_location.index[0]
t =  count_by_location.index[-1]
dummy = pd.DataFrame([[d, None] for d in pd.date_range(s, t)],columns=['date','dummy']).set_index('date')

count_by_location = dummy.join(count_by_location)
count_by_location = count_by_location.drop('dummy',axis=1)
count_by_location = count_by_location.fillna(0)

In [7]:
len(count_by_location) == len(dummy)

True

In [8]:
len(raw) == count_by_location.sum(axis=1).sum()

True

In [10]:
for region,prefs in region_to_prefecture.items():
    target = [e for e in prefs if e in count_by_location.columns]
    count_by_location[region] = count_by_location[target].sum(axis=1)

In [11]:
print(len(raw))
region_names = region_to_prefecture.keys()
print(count_by_location[region_names].sum(axis=1).sum())
print(count_by_location[set(count_by_location.columns) - set(prefectures) - set(region_names)].sum(axis=1).sum())

824
812.0
12.0


In [51]:
threshold = 5
res = {
    'original_date' : {},
}
for location in count_by_location.columns:
    res['original_date'][location] = {
        'daily_count': list(count_by_location[location]),
        'cumulative': list(count_by_location[location].cumsum()),
    }
res['date'] = [d.isoformat() for d in count_by_location[location].index]
res['prefectures'] = prefectures

def tuple2css(t):
    return f'rgb({t[0]}, {t[1]}, {t[2]})'

res['colors'] = {p: tuple2css(tableau20[i%20]) for i, p in enumerate(prefectures)}

In [52]:
with open('../../html/count_by_location.json', 'w') as f:
    json.dump(res, f)

In [44]:
prefs = region_to_prefecture['近畿']
target = [e for e in prefs if e in count_by_location.columns]
prefs, target
tmp = count_by_location['近畿'].cumsum()
# tmp = tmp[tmp>threshold]

In [45]:
tmp[tmp>threshold]

date
2020-02-13      6.0
2020-02-14      9.0
2020-02-15     10.0
2020-02-16     10.0
2020-02-17     13.0
2020-02-18     15.0
2020-02-19     15.0
2020-02-20     15.0
2020-02-21     15.0
2020-02-22     16.0
2020-02-23     16.0
2020-02-24     16.0
2020-02-25     16.0
2020-02-26     18.0
2020-02-27     21.0
2020-02-28     21.0
2020-02-29     21.0
2020-03-01     22.0
2020-03-02     24.0
2020-03-03     29.0
2020-03-04     38.0
2020-03-05     41.0
2020-03-06     58.0
2020-03-07     68.0
2020-03-08     81.0
2020-03-09     85.0
2020-03-10    109.0
2020-03-11    128.0
2020-03-12    146.0
2020-03-13    160.0
2020-03-14    176.0
2020-03-15    184.0
2020-03-16    190.0
2020-03-17    200.0
2020-03-18    211.0
2020-03-19    215.0
Name: 近畿, dtype: float64

In [43]:
tmp 

date
2020-01-15    0.0
2020-01-16    0.0
2020-01-17    0.0
2020-01-18    0.0
2020-01-19    0.0
             ... 
2020-03-15    4.0
2020-03-16    4.0
2020-03-17    4.0
2020-03-18    4.0
2020-03-19    4.0
Name: 東北, Length: 65, dtype: float64